<a href="https://colab.research.google.com/github/sbooeshaghi/BMGP_2020/blob/main/analysis/notebooks/10xFB/10xFB_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!date

Wed Jun 16 21:28:18 UTC 2021


# Install packages and download data

In [ ]:
import sys
COLAB = "google.colab" in sys.modules


In [ ]:
if COLAB:
    !git clone https://github.com/sbooeshaghi/BMGP_2020.git

Cloning into 'BMGP_2020'...
remote: Enumerating objects: 689, done.
remote: Counting objects: 100% (393/393), done.
remote: Compressing objects: 100% (342/342), done.
remote: Total 689 (delta 157), reused 186 (delta 48), pack-reused 296
Receiving objects: 100% (689/689), 469.81 MiB | 6.08 MiB/s, done.
Resolving deltas: 100% (248/248), done.
Checking out files: 100% (144/144), done.


In [ ]:
if COLAB:
    !pip install --quiet \
    kb-python==0.26.3 \
    anndata==0.7.6

     |████████████████████████████████| 59.1MB 71kB/s 
     |████████████████████████████████| 133kB 34.3MB/s 
     |████████████████████████████████| 10.3MB 21.5MB/s 
     |████████████████████████████████| 13.2MB 154kB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 112kB 52.3MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 1.2MB 32.2MB/s 


In [ ]:
if COLAB:
    # download cellranger (30sec)
    !wget -O cellranger-6.0.1.tar.gz \
    -q --show-progress --progress=bar:force  \
    "https://caltech.box.com/shared/static/i9dwsh2h6kojewjk967sjffdkunsk5tj.gz" \
    2>&1
    
    !tar -xf cellranger-6.0.1.tar.gz

    
    # download cellranger reference (5min)
    !wget \
    -q --show-progress --progress=bar:force  \
    https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-GRCh38-2020-A.tar.gz \
    2>&1
    
    !tar -xf refdata-gex-GRCh38-2020-A.tar.gz


cellranger-6.0.1.ta 100%[===================>]   1.01G  20.3MB/s    in 46s     
refdata-gex-GRCh38- 100%[===================>]  10.66G  38.5MB/s    in 4m 47s  


In [ ]:
%%bash
source cellranger-6.0.1/sourceme.bash

In [ ]:
if COLAB:
    # download the relevant data
    !wget -O 10xFB_5k_pbmc_v3_fastqs.tar.gz \
    -q --show-progress --progress=bar:force  \
    https://caltech.box.com/shared/static/z28rkegt9rdrmjlly9ashv7jqtghztjp.gz \
    2>&1
    !tar -xf 10xFB_5k_pbmc_v3_fastqs.tar.gz

10xFB_5k_pbmc_v3_fa 100%[===================>]   2.60G  18.1MB/s    in 2m 13s  


# Preprocess with kb

In [ ]:
%%time
!FB="BMGP_2020/references/10xFB/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/10xFB/kite/features.idx" && \
F2B="BMGP_2020/references/10xFB/kite/f2b.txt" && \
FASTA="BMGP_2020/references/10xFB/kite/features.fa" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
$FB

[2021-06-14 22:18:51,938]    INFO Generating mismatch FASTA at BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.fa
[2021-06-14 22:18:51,969]    INFO Creating transcript-to-gene mapping at BMGP_2020/references/10xFB_5k_pbmc_v3/kite/f2b.txt
[2021-06-14 22:18:51,976]    INFO Skipping kallisto index because BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.idx already exists. Use the --overwrite flag to overwrite.
CPU times: user 27 ms, sys: 7.25 ms, total: 34.2 ms
Wall time: 3.85 s


In [ ]:
%%bash
FB="BMGP_2020/references/10xFB/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/10xFB/kite/features.idx" && \
F2B="BMGP_2020/references/10xFB/kite/f2b.txt" && \
FASTA="BMGP_2020/references/10xFB/kite/features.fa" && \
OUT="kite_out" && \
FASTQS=$(FASTQDIR="fastqs/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R1) \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R2)) && \
/usr/bin/time --output=memtime_0.log -v kb count \
-i $INDEX \
-g $F2B \
-x 10xv3 \
-o $OUT \
--workflow kite:10xFB \
--h5ad \
--filter bustools \
$FASTQS

[2021-06-30 15:29:12,825]    INFO [count] Using index BMGP_2020/references/10xFB/kite/features.idx to generate BUS file to kite_out from
[2021-06-30 15:29:12,826]    INFO [count]         fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R1_001.fastq.gz
[2021-06-30 15:29:12,826]    INFO [count]         fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R2_001.fastq.gz
[2021-06-30 15:29:12,826]    INFO [count]         fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R1_001.fastq.gz
[2021-06-30 15:29:12,826]    INFO [count]         fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R2_001.fastq.gz
[2021-06-30 15:30:39,142]    INFO [count] Sorting BUS file kite_out/output.bus to kite_out/tmp/output.s.bus
[2021-06-30 15:30:57,579]    INFO [count] Whitelist not provided
[2021-06-30 15:30:57,580]    INFO [count] Copying pre-packaged 10XV3 whitelist to kite_out
[2021-06-30 15:30:58,444]    INFO [count] Copying 10xv3 feature-to-barcode map to kite_out
[2021-06-30 15:30:59,299]    INFO [count] Projecting BUS file kite_out/t

In [ ]:
!cat kite_out/run_info.json

{
	"n_targets": 0,
	"n_bootstraps": 0,
	"n_processed": 39096673,
	"n_pseudoaligned": 37262835,
	"n_unique": 37262835,
	"p_pseudoaligned": 95.3,
	"p_unique": 95.3,
	"kallisto_version": "0.46.2",
	"index_version": 0,
	"start_time": "Wed Jun 30 15:29:12 2021",
	"call": "/home/sina/bin/miniconda3/lib/python3.7/site-packages/kb_python/bins/linux/kallisto/kallisto bus -i BMGP_2020/references/10xFB/kite/features.idx -o kite_out -x 10xv3 -t 8 fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R1_001.fastq.gz fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R2_001.fastq.gz fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R1_001.fastq.gz fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R2_001.fastq.gz"
}


# Preprocess with cellranger

In [ ]:
!printf "fastqs,sample,library_type\n$PWD/fastqs,5k_pbmc_protein_v3_antibody,Antibody Capture" > BMGP_2020/references/10xFB/5k_pbmc_protein_v3_libraries.csv

In [ ]:
!./cellranger-6.0.1/cellranger

cellranger cellranger-6.0.1
Process 10x Genomics Gene Expression, Feature Barcode, and Immune Profiling data

USAGE:
    cellranger <SUBCOMMAND>

FLAGS:
    -h, --help       Prints help information
    -V, --version    Prints version information

SUBCOMMANDS:
    count               Count gene expression (targeted or whole-
                        transcriptome) and/or feature barcode reads
                        from a single sample and GEM well
    multi               Analyze multiplexed data or combined gene
                        expression/immune profiling/feature barcode data
    vdj                 Assembles single-cell VDJ receptor sequences from
                        10x Immune Profiling libraries
    aggr                Aggregate data from multiple Cell Ranger runs
    reanalyze           Re-run secondary analysis (dimensionality reduction,
                        clustering, etc)
    targeted-compare    Analyze targeted enrichment performance by comparing
               

In [ ]:
!LIB="BMGP_2020/references/10xFB/5k_pbmc_protein_v3_libraries.csv" && \
FR="BMGP_2020/references/10xFB/5k_pbmc_protein_v3_feature_ref.csv" && \
/usr/bin/time --output=memtime_1.log -v ./cellranger-6.0.1/cellranger count \
--libraries $LIB \
--feature-ref $FR \
--id count \
--nosecondary \
--disable-ui \
--expect-cells 5000 \
--transcriptome refdata-gex-GRCh38-2020-A

Martian Runtime - v4.0.4
Running preflight checks (please wait)...
Checking feature definition file...
Checking sample info...
Checking FASTQ folder...
Checking reference...
Checking reference_path (/home/sina/projects/kite/BMGP_2020/analysis/notebooks/10xFB/refdata-gex-GRCh38-2020-A) on atlas...
Checking optional arguments...
mrc: v4.0.4

mrp: v4.0.4

Anaconda: Python 3.7.6

numpy: 1.19.2

scipy: 1.1.0

pysam: 0.16.0.1

h5py: 2.8.0

pandas: 0.24.2

STAR: 2.7.2a

samtools: samtools 1.10
Using htslib 1.10.2
Copyright (C) 2019 Genome Research Ltd.

2021-06-30 15:38:55 [runtime] (ready)           ID.count.SC_RNA_COUNTER_CS.WRITE_GENE_INDEX
2021-06-30 15:38:55 [runtime] (run:local)       ID.count.SC_RNA_COUNTER_CS.WRITE_GENE_INDEX.fork0.chnk0.main
2021-06-30 15:38:55 [runtime] (ready)           ID.count.SC_RNA_COUNTER_CS.SC_MULTI_CORE.MAKE_FULL_CONFIG._MAKE_VDJ_CONFIG
2021-06-30 15:38:55 [runtime] (run:local)       ID.count.SC_RNA_COUNTER_CS.SC_MULTI_CORE.MAKE_FULL_CONFIG._MAKE_VDJ_CONFIG.